In [1]:
# سلول 0: وارد کردن کتابخانه‌های پایه ضروری
# توضیح: در این سلول، کتابخانه‌های مورد نیاز برای پردازش داده، تقسیم داده‌ها، و ارزیابی مدل را بارگذاری می‌کنیم.
import sys
import pandas as pd           # برای کار با داده‌های جدولی
import numpy as np            # برای محاسبات عددی
from sklearn.model_selection import train_test_split    # برای تقسیم داده به آموزش و تست
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # متریک‌های ارزیابی مدل

print("کتابخانه‌های لازم با موفقیت بارگذاری شدند.")


کتابخانه‌های لازم با موفقیت بارگذاری شدند.


In [4]:
# سلول 1: اضافه کردن مسیر مدل به sys.path برای ایمپورت کدهای سفارشی
# توضیح: این سلول مسیر فولدر مدل WeightedDecisionTreeModel را به sys.path اضافه می‌کند تا بتوانیم کلاس‌ها و توابع نوشته شده در فولدر مدل راوارد کنیم.

import sys
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src\model")
print("پوشه model به sys.path اضافه شد.")


پوشه model به sys.path اضافه شد.


In [5]:
# سلول 2: بارگذاری داده و نمایش نمونه‌ها
# توضیح: این سلول برای بارگذاری دیتافریم آماده شده از مسیر مشخص شده است و چند سطر اول داده را نشان می‌دهد.

data_path = r"C:\Users\HsH-HsH\weighted_split_criteria\notebooks\datasets\customer_churn_data_preprocessed.csv"

# بارگذاری فایل CSV به دیتافریم pandas
df = pd.read_csv(data_path)

print("داده مشتری با موفقیت بارگذاری شد. نمونه‌ای از داده:")
display(df.head())  # نمایش 5 نمونه اول به صورت جدول


داده مشتری با موفقیت بارگذاری شد. نمونه‌ای از داده:


,Age,LastTransaction,Churn,Gender_female,Gender_male,Payment Method_cash,Payment Method_cheque,Payment Method_credit card
0,64.0,98,1,False,True,False,False,True
1,35.0,118,0,False,True,False,True,False
2,25.0,107,1,True,False,False,False,True
3,39.0,90,1,False,True,False,False,True
4,28.0,189,0,True,False,False,True,False


In [6]:
# سلول 3: آماده‌سازی داده برای مدل - انتخاب ویژگی‌ها و ستون هدف و تقسیم داده
# توضیح: در این سلول ستون هدف (y) مشخص شده و بقیه ستون‌ها به عنوان ویژگی انتخاب می‌شوند.
# سپس داده به دو بخش آموزش (train) و تست (test) تقسیم می‌شود.

# ستون هدف (فرض می‌کنیم نام ستون 'Churn' است، اگر نامش فرق دارد بگو)
target_col = 'Churn'  

# ویژگی‌ها: حذف ستون هدف از کل ستون‌ها
X = df.drop(columns=[target_col])

# برچسب هدف
y = df[target_col]

# تقسیم داده به آموزش و تست، 80 درصد آموزش و 20 درصد تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"اندازه داده آموزش: {X_train.shape}")
print(f"اندازه داده تست: {X_test.shape}")


اندازه داده آموزش: (716, 7)
اندازه داده تست: (180, 7)


In [9]:
# سلول 4: تبدیل ستون‌های غیرعددی به عددی با one-hot encoding و همسان سازی ستون‌ها بین آموزش و تست
print("انواع داده ستون‌ها پیش از تبدیل:")
print(X_train.dtypes)

X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

print("\nانواع داده ستون‌ها پس از تبدیل:")
print(X_train_encoded.dtypes)

print(f"\nابعاد داده آموزش پس از تبدیل: {X_train_encoded.shape}")
print(f"ابعاد داده تست پس از تبدیل: {X_test_encoded.shape}")


انواع داده ستون‌ها پیش از تبدیل:
Age                           float64
LastTransaction                 int64
Gender_female                    bool
Gender_male                      bool
Payment Method_cash              bool
Payment Method_cheque            bool
Payment Method_credit card       bool
dtype: object

انواع داده ستون‌ها پس از تبدیل:
Age                           float64
LastTransaction                 int64
Gender_female                    bool
Gender_male                      bool
Payment Method_cash              bool
Payment Method_cheque            bool
Payment Method_credit card       bool
dtype: object

ابعاد داده آموزش پس از تبدیل: (716, 7)
ابعاد داده تست پس از تبدیل: (180, 7)


In [10]:
# سلول 5: تبدیل داده‌های آماده شده به numpy ndarray
X_train_array = X_train_encoded.values
X_test_array = X_test_encoded.values
y_train_array = y_train.values
y_test_array = y_test.values

print(f"نوع داده X_train: {type(X_train_array)}, شکل: {X_train_array.shape}")
print(f"نوع داده y_train: {type(y_train_array)}, شکل: {y_train_array.shape}")


نوع داده X_train: <class 'numpy.ndarray'>, شکل: (716, 7)
نوع داده y_train: <class 'numpy.ndarray'>, شکل: (716,)


In [12]:
# سلول جدید: اضافه کردن مسیرهای لازم به sys.path برای رفع ارور ماژول‌های پروژه
import sys
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src")
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src\model")
print("مسیرهای src و model به sys.path اضافه شدند.")


مسیرهای src و model به sys.path اضافه شدند.


In [13]:
# سلول 6: ایمپورت کلاس مدل WeightedDecisionTreeModel از پوشه مدل
# توضیح: این سلول کلاس مدل را از ماژول مدل سفارشی که به sys.path اضافه کردیم وارد می‌کند.

from weighted_decision_tree import WeightedDecisionTreeModel  # نام فایل و کلاس را با نام واقعی جایگزین کن
print("کلاس WeightedDecisionTreeModel با موفقیت ایمپورت شد.")


کلاس WeightedDecisionTreeModel با موفقیت ایمپورت شد.


In [14]:
import sys
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src")


In [2]:
# سلول جامع: آماده‌سازی داده‌ها، تبدیل نوع، ساخت و آموزش مدل WeightedDecisionTreeModel

import sys
import pandas as pd
import numpy as np

# اضافه کردن مسیر src به sys.path برای دسترسی به ماژول‌ها
sys.path.insert(0, r"C:\Users\HsH-HsH\weighted_split_criteria\src")

# بارگذاری داده
data_path = r"C:\Users\HsH-HsH\weighted_split_criteria\notebooks\datasets\customer_churn_data_preprocessed.csv"
df = pd.read_csv(data_path)

# تعریف ستون هدف
target_col = 'Churn'

# جداسازی ویژگی‌ها و هدف
X = df.drop(columns=[target_col])
y = df[target_col]

# تقسیم داده به آموزش و تست
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# تبدیل داده‌های غیرعددی به عددی با one-hot encoding
X_train_encoded = pd.get_dummies(X_train, drop_first=True)
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# اطمینان از همسانی ستون‌ها بین آموزش و تست
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# تبدیل به numpy array با نوع float برای جلوگیری از ارور np.isnan
X_train_array = X_train_encoded.values.astype(float)
X_test_array = X_test_encoded.values.astype(float)
y_train_array = y_train.values
y_test_array = y_test.values

print(f"اندازه داده آموزش: {X_train_array.shape}, نوع داده: {X_train_array.dtype}")
print(f"اندازه داده تست: {X_test_array.shape}, نوع داده: {X_test_array.dtype}")

# ایمپورت کلاس مدل
from weighted_decision_tree import WeightedDecisionTreeModel

# ساخت مدل با داده ویژگی‌های کامل و عمق ماکزیمم ۵
model = WeightedDecisionTreeModel(
    X_data=X_train_array,
    max_depth=5
)

# آموزش مدل
model.fit(X_train_array, y_train_array)

print("مدل WeightedDecisionTree با موفقیت ساخته و آموزش داده شد.")


اندازه داده آموزش: (716, 7), نوع داده: float64
اندازه داده تست: (180, 7), نوع داده: float64


ModuleNotFoundError: No module named 'weighted_decision_tree'